In [ ]:
## First import packages
import numpy as np
import tensorflow as tf
import keras
from keras import Model
import os, glob
import resnet_modified # modified to have zero verbosity, deperacated several printing step, lr -> learning_rate in calling adam.
from keras.datasets import cifar10
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from resnet_modified import ResNet
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.models import load_model
from keras import layers
from keras.layers import Dense
import pandas as pd

## Define local labeling dictionary and functions
## Function: learning rate schedule
## This is used in specialization method
## Fundtion: Specialization
## This is method of logifold. Customized here adopting local load and usage.
label_dict = {(3,1) : 'ResNet20v1',
        (3,2) : 'ResNet20v2',
        (9,1): 'ResNet56v1',
        (9,2): 'ResNet56v2'}

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 5, 10, 15, 18 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 18:
        lr *= 0.5e-3
    elif epoch > 15:
        lr *= 1e-3
    elif epoch > 10:
        lr *= 1e-2
    elif epoch > 5:
        lr *= 1e-1
#     print('Learning rate: ', lr)
    return lr

def turn_specialist(model : Model, path : str,
        x_tr: np.ndarray | None = None,
        y_tr: np.ndarray | None = None,
        x_v: np.ndarray | None = None,
        y_v: np.ndarray | None = None,
        epochs: int = 21,
        learning_rate : float = 1e-3,
        batch_size: int = 128,
        save_each: bool = False,
        save_bests: int | None = None,
        verbose: int = 1,
    ):
        
        # build specialist network
        base = Model(inputs = model.inputs, outputs = model.layers[-2].output, name="base")
        x    = keras.Input(shape=base.input_shape[1:], name="in")
        y    = Dense(10, name="dense")(base(x)) # 10 can be changed to len(newtarget)
        z    = layers.Softmax(name="softmax")(y)
        specialist = Model(inputs = x, outputs = z)
        specialist.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )



        # callbacks
        callbacks = [ModelCheckpoint(path, monitor="val_accuracy",
                                    save_best_only=True, verbose=verbose)]
        
        callbacks += [LearningRateScheduler(lr_schedule),
                        ReduceLROnPlateau(factor=np.sqrt(0.1), patience=5, min_lr=5e-7)]

        # fit 
        hist = specialist.fit(x_tr, y_tr, batch_size=batch_size,
                        validation_data=(x_v, y_v),
                        epochs=epochs, callbacks=callbacks, verbose=verbose)



        # ---------- summary ----------
        metric = "val_accuracy"
        best = np.max(hist.history[metric])
        first = hist.history[metric][0]
        print(f"best {metric} {best:.3f} (first {first:.3f})")
    
## Load data and preprocess a bit.
(x, y), (x_test, y_test) = cifar10.load_data()
x_train, x_v, y_train, y_v = train_test_split(x, y, test_size=0.2, 
                                              random_state=42) # random state has been always 42.
x_train = x_train.astype('float32') / 255.0
x_val = x_v.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train_categorical_10 = to_categorical(y_train,10)
y_val_categorical_10 = to_categorical(y_v,10)
y_test_categorical_10 = to_categorical(y_test,10)
y_test_categorical_20 = to_categorical(y_test,20)

training_y_long=np.concatenate([y_train,y_train+10],axis=0)
training_y_long = keras.utils.to_categorical(training_y_long,20)
validating_y_long=np.concatenate([y_v,y_v+10],axis=0)
validating_y_long = keras.utils.to_categorical(validating_y_long,20)

In [2]:
## Define custom function for generating result table.

def make_df(results:dict, single:bool = False):
    rows = []
    if not single:
        
        for (structure, attack_type), eval_dict in results.items():
            accs = [eval_dict.get(f'acc_{i}', np.nan) for i in range(len(eval_dict))]
            accs.sort(reverse = True)
            acc_av = np.nanmean(accs)
            losses = [eval_dict.get(f'loss_{i}', np.nan) for i in range(len(eval_dict))]
            losses.sort()
            losses_av = np.nanmean(losses)
            row = [f'{structure}']+[f'{attack_type}'] + [acc_av] + [losses_av]
            rows.append(row)
    else:
        for (structure, attack_type), acc_dict in results.items():
            accs = [acc_dict.get(f'acc_0', np.nan)]
            losses = [acc_dict.get(f'loss_0', np.nan)]
            row = [f'{structure}']+[f'{attack_type}'] + accs + losses
            rows.append(row)

    return pd.DataFrame(rows, columns=['model','attack_type', 'acc_av', 'loss_av'])

def make_row(structure : str,attack_and_model_type : str, model : keras.Model,
             idx : int = 0, results : dict = {},
             x = x_test, name_of_testing_sample : str = 'Original',
             y = y_test.reshape(-1),
            verbose = 1):
    key = (structure, attack_and_model_type)
    loss, acc = model.evaluate(x, y, verbose = verbose)
    
    if key not in results:
        results[key] = {}
    results[key][f'acc_{idx}_on_{name_of_testing_sample}'] = acc
    if name_of_testing_sample == 'Original':
        results[key][f'loss_{idx}_on_Original'] = loss
    else:
        pass
    return results

### $M_{D\sqcup \widetilde{D}}$

$\widetilde{D} \in \{\textrm{untargeted PGD}, \textrm{targeted PGD}, \textrm{CWL2}\}$


In [28]:
folder = 'adversarial_models/full_and_fine/'
pattern = os.path.join(folder, '*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}

results = {}  # structure: {(structure, attack type): [acc_0, acc_1, acc_2, acc_3]}

for filename, model in loaded_models.items():
    # Example filename: 'ResNet20v1_ff_untargeted-adv_0.keras'
    # Extract family and index
    base = filename.replace('.keras','')
    parts = base.split('_')
    structure = parts[0]  # ResNet20v1, etc.
    attack_and_model_type = parts[2] + '_ff' 
    idx = int(parts[-1].split('-')[-1])  # last digit after -adv_

    results = make_row(structure, attack_and_model_type, model, 
            idx = idx,results = results,
                      y = y_test_categorical_20)

df = make_df(results)
df.to_csv('results/result_full_and_fine_testing_original.csv')

313/313 [==============================] - 2s 5ms/step - loss: 0.9614 - acc: 0.7914


In [29]:
df

,model,attack_type,acc_av,loss_av
0,ResNet20v1,cw-adv_ff,0.696200,1.414593
1,ResNet20v1,targeted-adv_ff,0.855550,0.707468
2,ResNet20v1,untargeted-adv_ff,0.855525,0.704581
3,ResNet20v2,cw-adv_ff,0.729950,1.221137
4,ResNet20v2,targeted-adv_ff,0.819550,0.875817
5,ResNet20v2,untargeted-adv_ff,0.804600,0.899692
6,ResNet56v1,cw-adv_ff,0.740500,1.299252
7,ResNet56v1,targeted-adv_ff,0.840400,0.869057
8,ResNet56v1,untargeted-adv_ff,0.838750,0.837809
9,ResNet56v2,cw-adv_ff,0.766500,1.053291


### $\widetilde{M}_{D\sqcup \widetilde{D}}$

$\widetilde{D} \in \{\textrm{untargeted PGD}, \textrm{targeted PGD}, \textrm{CWL2}\}$


In [30]:
folder = 'adversarial_models/specialized_from_full_and_fine/'
pattern = os.path.join(folder, '*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}

results = {}  # structure: {(structure, attack type): [acc_0, acc_1, acc_2, acc_3]}

for filename, model in loaded_models.items():
    # Example filename: 'classifying_original_from_ResNet20v1_ff_cw-adv_0.keras'
    # Extract family and index
    base = filename.replace('.keras','')
    parts = base.split('_')
    structure = parts[3]  # ResNet20v1, etc.
    attack_and_model_type = parts[5] + '_experts'
    idx = int(parts[-1].split('-')[-1])  # last digit after -adv_

    results = make_row(structure, attack_and_model_type, model, 
            idx = idx,results = results,
                      y = y_test_categorical_10)

df = make_df(results)
df.to_csv('results/result_specialized_from_ff_testing_original.csv')

313/313 [==============================] - 6s 6ms/step - loss: 0.4349 - accuracy: 0.9118


In [31]:
df

,model,attack_type,acc_av,loss_av
0,ResNet20v1,cw-adv_experts,0.903150,0.507745
1,ResNet20v1,targeted-adv_experts,0.902950,0.483714
2,ResNet20v1,untargeted-adv_experts,0.901575,0.498358
3,ResNet20v2,cw-adv_experts,0.905975,0.501167
4,ResNet20v2,targeted-adv_experts,0.908300,0.473372
5,ResNet20v2,untargeted-adv_experts,0.906775,0.480066
6,ResNet56v1,cw-adv_experts,0.911700,0.513016
7,ResNet56v1,targeted-adv_experts,0.909075,0.481757
8,ResNet56v1,untargeted-adv_experts,0.909850,0.474145
9,ResNet56v2,cw-adv_experts,0.914600,0.483012


### $M_{D\cup \widetilde{D}}$ and $\widetilde{M}_{D\cup \widetilde{D}}$


$\widetilde{D} \in \{\textrm{untargeted PGD}, \textrm{targeted PGD}, \textrm{CWL2}\}$


In [32]:
folder = 'adversarial_models/naive_method/'
pattern = os.path.join(folder, '*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}

results_naive = {}  
results_fine = {}
for filename, model in loaded_models.items():
    # Example filename: 'ResNet20v1_naive_untargeted.keras'
    # or 'ResNet20v1_naive_untargeted_fine.keras'
    # Extract family and index
    base = filename.replace('.keras','')
    parts = base.split('_')
    structure = parts[0]  # ResNet20v1, etc.
    if parts[-1] == 'fine':
        attack_and_model_type = parts[-2] + '_naive_specialized_once'
        results_fine = make_row(structure, attack_and_model_type, model, 
    #             idx = idx,
                           results = results_fine,
                          y = y_test_categorical_10)
    else:
        attack_and_model_type = parts[-1] + '_naive'
    #     idx = int(parts[-1].split('-')[-1])  # last digit after -adv_

        results_naive = make_row(structure, attack_and_model_type, model, 
    #             idx = idx,
                           results = results_naive,
                          y = y_test_categorical_10)

df_naive = make_df(results_naive)
df_naive_fine = make_df(results_fine)
df_naive.to_csv('results/result_naive_testing_original.csv')
df_naive_fine.to_csv('results/result_naive_fine_testing_original.csv')
df_naive

313/313 [==============================] - 3s 6ms/step - loss: 0.4436 - accuracy: 0.9101


,model,attack_type,acc_av,loss_av
0,ResNet20v1,cw_naive,0.8480,0.776990
1,ResNet20v1,targeted_naive,0.8424,0.828501
2,ResNet20v1,untargeted_naive,0.8232,0.965299
3,ResNet20v2,cw_naive,0.8146,0.872845
4,ResNet20v2,targeted_naive,0.7994,1.054251
5,ResNet20v2,untargeted_naive,0.8040,0.945313
6,ResNet56v1,cw_naive,0.8807,0.662840
7,ResNet56v1,targeted_naive,0.8437,0.890810
8,ResNet56v1,untargeted_naive,0.8513,0.808957
9,ResNet56v2,cw_naive,0.8171,0.876579


In [33]:
df_naive_fine

,model,attack_type,acc_av,loss_av
0,ResNet20v1,cw_naive_specialized_once,0.9073,0.457576
1,ResNet20v1,targeted_naive_specialized_once,0.9016,0.469850
2,ResNet20v1,untargeted_naive_specialized_once,0.9051,0.473326
3,ResNet20v2,cw_naive_specialized_once,0.9070,0.492664
4,ResNet20v2,targeted_naive_specialized_once,0.9016,0.479983
5,ResNet20v2,untargeted_naive_specialized_once,0.9069,0.473368
6,ResNet56v1,cw_naive_specialized_once,0.9108,0.512450
7,ResNet56v1,targeted_naive_specialized_once,0.9074,0.498648
8,ResNet56v1,untargeted_naive_specialized_once,0.9056,0.477092
9,ResNet56v2,cw_naive_specialized_once,0.9176,0.435703


### Standard CIFAR10 models and its specialization (once) on original testing dataset 

In [24]:
folder = 'CIFAR10models/ResNet/'
pattern = os.path.join(folder, '*cifar10*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}

results = {}  

for filename, model in loaded_models.items():
    # Example filename: 'n_3_v1_cifar10_2.keras'
    # Extract family and index
    base = filename.replace('.keras','')
    parts = base.split('_')
    structure = label_dict[(int(parts[1]),int(parts[2][-1]))]  # ResNet20v1, etc.
    attack_and_model_type = 'Standard'
    idx = int(parts[-1][-1])  # possibly cifar10 -> 0

    results = make_row(structure, attack_and_model_type, model, 
            idx = idx,results = results,
                      y = y_test_categorical_10)

df = make_df(results)
df.to_csv('results/result_standard-models_testing_original.csv')

313/313 [==============================] - 2s 5ms/step - loss: 0.8390 - acc: 0.8241


In [25]:
folder = 'CIFAR10models/more_tunned/'
pattern = os.path.join(folder, '*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}

results = {}  

for filename, model in loaded_models.items():
    # Example filename: 'ResNet20v1_more_specialized_1.keras'
    # Extract family and index
    base = filename.replace('.keras','')
    parts = base.split('_')
    if parts[1] != 'more':
        continue
    structure = parts[0]  # ResNet20v1, etc.
    attack_and_model_type = 'once_specialized_from_standard'
    idx = int(parts[-1])  

    results = make_row(structure, attack_and_model_type, model, 
            idx = idx,results = results,
                      y = y_test_categorical_10)

df = make_df(results)
df.to_csv('results/result_once_specialized_from_standard-models_testing_original.csv')

313/313 [==============================] - 3s 5ms/step - loss: 0.4540 - accuracy: 0.9158


## Testing on other dataset

PGD type, targeted or untargeted, generated by resnet and vgg models

cwl2 type, targeted or untargeted, generated by resnet models



In [3]:
## set all files.
## Possibly there is no merged testing dataset, or validation dataset.
## We only look for testing dataset.
import re

pattern1 = re.compile(r'_\d+to\d+_')
pattern2 = re.compile(r'test')

folder = './adversarial_examples/gen_by_ResNet'
all_files = os.listdir(folder)

filtered_files = [
    fname for fname in all_files
    if fname.endswith('.npy') and not pattern1.search(fname) and pattern2.search(fname)
]




In [4]:
## load all the testing dataset
## including testing dataset generated by VGG
loaded_testing_dataset = {}
import re
for f in filtered_files:
    
    base = f.replace('.keras.npy','').replace('.npy','')
    parts = base.split('_')
    attack_type = parts[0]
    if attack_type == 'cwl2':
        direction = parts[3]
        if direction == 'targeted':
            direction = direction +'_'+ parts[5]
        if parts[-1].isdigit():
            idx = 1
        else:
            idx = 0
        model_type = 'ResNet56v1'
    else:
        
        direction = parts[4]
        if direction == 'target':
            if parts[6] == 'second':
                direction = 'targeted_2nd'
                model_type = 'ResNet56v1'
                
            else:
                direction = 'targeted_ll'
                model_type = label_dict[(int(parts[10]),
                                        int(parts[11][-1]))]
                idx = parts[-1][-1]
        else:
            direction = 'untargeted'
            model_type = label_dict[(int(parts[8]),
                                        int(parts[9][-1]))]
            idx = parts[-1][-1]
    key = (attack_type, direction, model_type, idx)
    loaded_testing_dataset[key] = np.load(os.path.join(folder, f))


folder = './adversarial_examples/gen_by_VGG'
all_files = os.listdir(folder)

filtered_files = [
    fname for fname in all_files
    if fname.endswith('.npy') and pattern2.search(fname)
]
for f in filtered_files:
    base = f.replace('.npy','')
    parts = base.split('_')
    attack_type = parts[0]
    direction = parts[4]
    if direction == 'target':
        direction = 'targeted_ll'
        model_type = parts[8]
        if parts[-1].isdigit():
            idx = int(parts[-1])
        else:
            idx = 4
    else:
        direction = 'untargeted'
        model_type = parts[6]
        if parts[-1].isdigit():
            idx = int(parts[-1])
        else:
            idx = 4
    key = (attack_type, direction, model_type, idx)
    loaded_testing_dataset[key] = np.load(os.path.join(folder, f))

            
                 

In [5]:

def evaluate_model_on_datasets(model_id : tuple, model : keras.Model,  datasets : dict, 
                               y,
                               results : dict = {},
                               verbose=0):
    """
    model_id : tuple (structure, model type, adversarial dataset (if any. else none), index)
    datasets key: tuple (attack type, attack direction, generating model, index)
    results: nested dict to fill, key: (model_id, dataset_id)
    y : truth label
    """
    
    y = y
    for dataset_id, x in datasets.items():
        loss, acc = model.evaluate(x, y, verbose=verbose)
        print(dataset_id)
        results[(model_id, dataset_id)] = {'acc': acc, 'loss': loss}
        
    return results

def make_df(results):
    """
    results: dict with key: (model_id, dataset_id) -> {'acc': value, 'loss': value}
    Returns dataframe with models as rows, datasets as columns (acc or loss).
    """
    rows = []
    for (model_id, dataset_id), metrics in results.items():
        row = list(model_id) + list(dataset_id) + [metrics['acc'], metrics['loss']]
        rows.append(row)
    
    df = pd.DataFrame(rows, columns=['testing_model_structure', 'testing_model_type', 'testing_model_augmentation', 'testing_model_idx',
                                     'D_attack_type', 'D_attack_direction', 'D_generating_model', 'D_idx',
                                     'acc', 'loss'])
    return df


In [6]:
# List of folders and (optional) patterns to search for models
model_sources = [
    ('adversarial_models/full_and_fine/', '*.keras'),
    ('adversarial_models/specialized_from_full_and_fine/', '*.keras'),
    ('adversarial_models/naive_method/', '*.keras'),
    ('CIFAR10models/more_tunned/', '*.keras'),
    ('CIFAR10models/ResNet/', '*cifar10*.keras'),
]

# Initialize list of all found files
file_list = []

for folder, pattern in model_sources:
    full_pattern = os.path.join(folder, pattern)
    matched_files = sorted(glob.glob(full_pattern))
    file_list.extend(matched_files)

# # Load all models into dictionary: {filename: model}
# loaded_models = {os.path.basename(f): load_model(f) for f in file_list}

In [7]:
%%time
loaded_models = {}
for f in file_list:
    base = os.path.basename(f)
    parts = base.replace('.keras','').split('_')
    if parts[0] == 'classifying':
        model_structure = parts[3]
        model_type = 'S-ff'
        augmented_by = parts[5].split('-')[0]
        idx = parts[-1]
        model_key = (model_structure, model_type, augmented_by, idx)

    elif parts[0] == 'n':
        model_structure = label_dict[(int(parts[1]),
                                    int(parts[2][-1]))]
        model_type = 'B'
        augmented_by = 'None'
        idx = parts[-1][-1]
        model_key = (model_structure, model_type, augmented_by, idx)
    else:
        model_structure = parts[0]
        if parts[1] == 'ff':
            model_type = 'ff'
            augmented_by = parts[2].split('-')[0]
            idx = parts[-1]
            model_key = (model_structure, model_type, augmented_by, idx)
            
        elif parts[1] == 'naive':
            if parts[-1] == 'fine':
                model_type = 'S-nai'
            else:
                model_type = 'nai'
            augemented_by = parts[2]
            idx = 0
            model_key = (model_structure, model_type, augmented_by, idx)
        else:
            if parts[1] == 'more':
                model_type = 'S-B'
                augmented_by = 'None'
                idx = parts[-1]
                model_key = (model_structure, model_type, augmented_by, idx)
            else:
                continue
    
    loaded_models[model_key]=load_model(f)

CPU times: user 1min 49s, sys: 3.99 s, total: 1min 53s
Wall time: 2min 2s


In [8]:
loaded_testing_dataset[('original','original','original', '0')] = x_test

In [17]:
import pandas as pd
df1 = pd.read_csv('results/master_results1.csv')
df2 = pd.read_csv('results/master_results2.csv')
df3 = pd.read_csv('results/master_results3.csv')

In [34]:
df2.iloc[6286:]

,testing_model_structure,testing_model_type,testing_model_augmentation,testing_model_idx,D_attack_type,D_attack_direction,D_generating_model,D_idx,acc,loss
6286,ResNet56v1,B,None,2,pgd,untargeted,ResNet56v2,2,0.7847,1.813779
6287,ResNet56v1,B,None,2,pgd,targeted_ll,vgg19,1,0.5445,2.686844
6288,ResNet56v1,B,None,2,pgd,untargeted,vgg13,1,0.8212,1.480899
6289,ResNet56v1,B,None,2,pgd,targeted_ll,vgg19,0,0.5540,2.730550
6290,ResNet56v1,B,None,2,pgd,untargeted,vgg11,0,0.7974,1.338869
6291,ResNet56v1,B,None,2,pgd,untargeted,vgg19,2,0.8066,1.768321
6292,ResNet56v1,B,None,2,pgd,untargeted,vgg16,0,0.8145,1.842525


In [41]:
df3 = df3.iloc[52:]

In [49]:
df2

,testing_model_structure,testing_model_type,testing_model_augmentation,testing_model_idx,D_attack_type,D_attack_direction,D_generating_model,D_idx,acc,loss
0,ResNet20v1,S-ff,untargeted,3,cwl2,untargeted,ResNet56v1,1,0.2595,9.254398
1,ResNet20v1,S-ff,untargeted,3,pgd,targeted_ll,ResNet20v1,1,0.8405,0.717156
2,ResNet20v1,S-ff,untargeted,3,pgd,targeted_ll,ResNet56v2,2,0.8386,0.751812
3,ResNet20v1,S-ff,untargeted,3,pgd,targeted_ll,ResNet20v2,2,0.8230,0.809612
4,ResNet20v1,S-ff,untargeted,3,pgd,untargeted,ResNet20v2,3,0.8743,0.670368
...,...,...,...,...,...,...,...,...,...,...
6288,ResNet56v1,B,None,2,pgd,untargeted,vgg13,1,0.8212,1.480899
6289,ResNet56v1,B,None,2,pgd,targeted_ll,vgg19,0,0.5540,2.730550
6290,ResNet56v1,B,None,2,pgd,untargeted,vgg11,0,0.7974,1.338869
6291,ResNet56v1,B,None,2,pgd,untargeted,vgg19,2,0.8066,1.768321


In [51]:
df3

,testing_model_structure,testing_model_type,testing_model_augmentation,testing_model_idx,D_attack_type,D_attack_direction,D_generating_model,D_idx,acc,loss
52,ResNet56v1,B,None,2,pgd,targeted_ll,vgg16,1,0.5496,2.555589
53,ResNet56v1,B,None,2,pgd,untargeted,vgg13,4,0.8237,1.464711
54,ResNet56v1,B,None,2,pgd,untargeted,vgg16,2,0.8092,1.852720
55,ResNet56v1,B,None,2,pgd,targeted_ll,vgg16,0,0.5573,2.532015
56,ResNet56v1,B,None,2,pgd,untargeted,vgg16,1,0.8046,1.935211
...,...,...,...,...,...,...,...,...,...,...
469,ResNet56v2,B,None,3,pgd,targeted_ll,vgg13,0,0.5287,2.633854
470,ResNet56v2,B,None,3,pgd,untargeted,vgg16,4,0.7913,1.770648
471,ResNet56v2,B,None,3,pgd,untargeted,vgg19,0,0.8060,1.407976
472,ResNet56v2,B,None,3,pgd,targeted_ll,vgg11,0,0.6429,1.677405


In [59]:
df

,testing_model_structure,testing_model_type,testing_model_augmentation,testing_model_idx,D_attack_type,D_attack_direction,D_generating_model,D_idx,acc,loss
0,ResNet20v1,S-ff,untargeted,3,cwl2,untargeted,ResNet56v1,1,0.2595,9.254398
1,ResNet20v1,S-ff,untargeted,3,pgd,targeted_ll,ResNet20v1,1,0.8405,0.717156
2,ResNet20v1,S-ff,untargeted,3,pgd,targeted_ll,ResNet56v2,2,0.8386,0.751812
3,ResNet20v1,S-ff,untargeted,3,pgd,targeted_ll,ResNet20v2,2,0.8230,0.809612
4,ResNet20v1,S-ff,untargeted,3,pgd,untargeted,ResNet20v2,3,0.8743,0.670368
...,...,...,...,...,...,...,...,...,...,...
469,ResNet56v2,B,None,3,pgd,targeted_ll,vgg13,0,0.5287,2.633854
470,ResNet56v2,B,None,3,pgd,untargeted,vgg16,4,0.7913,1.770648
471,ResNet56v2,B,None,3,pgd,untargeted,vgg19,0,0.8060,1.407976
472,ResNet56v2,B,None,3,pgd,targeted_ll,vgg11,0,0.6429,1.677405


In [58]:
df_new = pd.read_csv('results/master_results23.csv')
df_new

,testing_model_structure,testing_model_type,testing_model_augmentation,testing_model_idx,D_attack_type,D_attack_direction,D_generating_model,D_idx,acc,loss
0,ResNet20v1,S-ff,untargeted,3,cwl2,untargeted,ResNet56v1,1,0.2595,9.254398
1,ResNet20v1,S-ff,untargeted,3,pgd,targeted_ll,ResNet20v1,1,0.8405,0.717156
2,ResNet20v1,S-ff,untargeted,3,pgd,targeted_ll,ResNet56v2,2,0.8386,0.751812
3,ResNet20v1,S-ff,untargeted,3,pgd,targeted_ll,ResNet20v2,2,0.8230,0.809612
4,ResNet20v1,S-ff,untargeted,3,pgd,untargeted,ResNet20v2,3,0.8743,0.670368
...,...,...,...,...,...,...,...,...,...,...
6710,ResNet56v2,B,None,3,pgd,targeted_ll,vgg13,0,0.5287,2.633854
6711,ResNet56v2,B,None,3,pgd,untargeted,vgg16,4,0.7913,1.770648
6712,ResNet56v2,B,None,3,pgd,untargeted,vgg19,0,0.8060,1.407976
6713,ResNet56v2,B,None,3,pgd,targeted_ll,vgg11,0,0.6429,1.677405


In [59]:
condition = ((df['testing_model_structure'] == 'ResNet20v1')
            &(df['testing_model_type'] == 'S-ff')
            &(df['testing_model_augmentation'] == 'untargeted')
            &(df['testing_model_idx'] == 3))

In [62]:
df1 = df.iloc[:4661]

In [65]:
df1.to_csv('results/master_results1.csv', index=False)

In [9]:
results = {}
y = y_test_categorical_10
for model_key, m in loaded_models.items():
    if model_key in [('ResNet56v1', 'B', 'None', '2'),
('ResNet56v1', 'B', 'None', '3'),
('ResNet56v2', 'B', 'None', '0'),
('ResNet56v2', 'B', 'None', '1'),
('ResNet56v2', 'B', 'None', '2'),
('ResNet56v2', 'B', 'None', '3')]:
        
        print(model_key)
        results = evaluate_model_on_datasets(model_key, m, loaded_testing_dataset, y,
                                            verbose = 0, results = results)

('ResNet56v1', 'B', 'None', '2')
('cwl2', 'untargeted', 'ResNet56v1', 1)
('pgd', 'targeted_ll', 'ResNet20v1', '1')
('pgd', 'targeted_ll', 'ResNet56v2', '2')
('pgd', 'targeted_ll', 'ResNet20v2', '2')
('pgd', 'untargeted', 'ResNet20v2', '3')
('pgd', 'targeted_ll', 'ResNet20v1', '0')
('pgd', 'untargeted', 'ResNet56v2', '1')
('pgd', 'targeted_ll', 'ResNet56v2', '1')
('pgd', 'targeted_ll', 'ResNet20v1', '6')
('pgd', 'untargeted', 'ResNet20v1', '5')
('pgd', 'untargeted', 'ResNet56v2', '3')
('pgd', 'untargeted', 'ResNet20v2', '2')
('pgd', 'targeted_ll', 'ResNet56v1', '3')
('pgd', 'targeted_ll', 'ResNet56v2', '3')
('pgd', 'targeted_ll', 'ResNet20v2', '3')
('pgd', 'untargeted', 'ResNet20v2', '1')
('pgd', 'targeted_ll', 'ResNet56v1', '1')
('pgd', 'untargeted', 'ResNet56v1', '3')
('pgd', 'untargeted', 'ResNet20v1', '7')
('pgd', 'targeted_2nd', 'ResNet56v1', '7')
('pgd', 'untargeted', 'ResNet20v1', '1')
('pgd', 'targeted_ll', 'ResNet56v1', '2')
('pgd', 'untargeted', 'ResNet56v1', '2')
('pgd', 'tar

('pgd', 'untargeted', 'vgg11', 0)
('pgd', 'untargeted', 'vgg19', 2)
('pgd', 'untargeted', 'vgg16', 0)
('pgd', 'targeted_ll', 'vgg16', 1)
('pgd', 'untargeted', 'vgg13', 4)
('pgd', 'untargeted', 'vgg16', 2)
('pgd', 'targeted_ll', 'vgg16', 0)
('pgd', 'untargeted', 'vgg16', 1)
('pgd', 'targeted_ll', 'vgg16', 2)
('pgd', 'untargeted', 'vgg11', 4)
('pgd', 'targeted_ll', 'vgg11', 1)
('pgd', 'targeted_ll', 'vgg19', 2)
('pgd', 'targeted_ll', 'vgg13', 4)
('pgd', 'untargeted', 'vgg19', 4)
('pgd', 'targeted_ll', 'vgg13', 1)
('pgd', 'targeted_ll', 'vgg13', 2)
('pgd', 'targeted_ll', 'vgg19', 4)
('pgd', 'untargeted', 'vgg11', 2)
('pgd', 'targeted_ll', 'vgg11', 2)
('pgd', 'targeted_ll', 'vgg16', 4)
('pgd', 'untargeted', 'vgg13', 0)
('pgd', 'targeted_ll', 'vgg11', 4)
('pgd', 'untargeted', 'vgg13', 2)
('pgd', 'untargeted', 'vgg11', 1)
('pgd', 'untargeted', 'vgg19', 1)
('pgd', 'targeted_ll', 'vgg13', 0)
('pgd', 'untargeted', 'vgg16', 4)
('pgd', 'untargeted', 'vgg19', 0)
('pgd', 'targeted_ll', 'vgg11', 0)


('pgd', 'targeted_ll', 'ResNet20v2', '1')
('pgd', 'targeted_ll', 'ResNet20v2', '0')
('pgd', 'untargeted', 'ResNet56v1', '1')
('pgd', 'targeted_ll', 'ResNet56v2', '0')
('pgd', 'targeted_ll', 'ResNet20v1', '5')
('pgd', 'targeted_ll', 'ResNet56v1', '0')
('pgd', 'targeted_ll', 'ResNet20v1', '2')
('pgd', 'untargeted', 'ResNet20v2', '0')
('pgd', 'untargeted', 'ResNet20v1', '6')
('pgd', 'targeted_ll', 'ResNet20v1', '4')
('cwl2', 'targeted_2nd', 'ResNet56v1', 0)
('cwl2', 'targeted_ll', 'ResNet56v1', 0)
('pgd', 'untargeted', 'ResNet20v1', '4')
('pgd', 'untargeted', 'ResNet56v2', '0')
('pgd', 'targeted_ll', 'ResNet20v1', '3')
('pgd', 'targeted_ll', 'ResNet20v1', '7')
('cwl2', 'targeted_ll', 'ResNet56v1', 1)
('pgd', 'untargeted', 'ResNet20v1', '0')
('pgd', 'untargeted', 'ResNet56v1', '0')
('pgd', 'untargeted', 'ResNet20v1', '3')
('pgd', 'untargeted', 'ResNet20v1', '2')
('cwl2', 'untargeted', 'ResNet56v1', 0)
('pgd', 'untargeted', 'ResNet56v2', '2')
('pgd', 'targeted_ll', 'vgg19', 1)
('pgd', 'unta

In [10]:
df3 = make_df(results)

In [11]:
df3.to_csv('results/master_results3.csv')

In [88]:
len(loaded_models)*len(loaded_testing_dataset)

11376

In [38]:
## Make a result dataframe.
## It takes a lot of time.

folder = 'adversarial_models/full_and_fine/'
pattern = os.path.join(folder, '*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}

results = {}  # structure: {(structure, attack type): [acc_0, acc_1, acc_2, acc_3]}

for filename, model in loaded_models.items():
    ###################
    if filename.startswith('ResNet20v1_ff_targeted'):
        print(filename)
    ###############

        # Example filename: 'ResNet20v1_ff_untargeted-adv_0.keras'
        # Extract family and index
        base = filename.replace('.keras','')
        parts = base.split('_')
        structure = parts[0]  # ResNet20v1, etc.
        attack_and_model_type = parts[2] + '_ff' 
        idx = int(parts[-1].split('-')[-1])  # last digit after -adv_
        testing_sample = x_test
        results = make_row(structure, attack_and_model_type, model,
                           idx = idx,results = results,
                           x = testing_sample,
                           y = y_test_categorical_20) # ff -> 20 dimensional answer
        for k, testing_sample in loaded_testing_dataset.items():
            name_of_testing_sample = 'adv_'+'_'.join(k[:-1])+'_'+str(k[-1]) # 'adv_pgd_targeted_ll_vgg11_0'
            results = make_row(structure, attack_and_model_type, model,
                               idx = idx,results = results,
                               x = testing_sample,
                               name_of_testing_sample = name_of_testing_sample,
                               y = y_test_categorical_20,
                               verbose = 0)

ResNet20v1_ff_targeted-adv_0.keras
313/313 [==============================] - 16s 2ms/step - loss: 0.8025 - acc: 0.8452
ResNet20v1_ff_targeted-adv_1.keras
313/313 [==============================] - 1s 2ms/step - loss: 0.8304 - acc: 0.8357
ResNet20v1_ff_targeted-adv_2.keras
313/313 [==============================] - 1s 2ms/step - loss: 0.6017 - acc: 0.8722
ResNet20v1_ff_targeted-adv_3.keras
313/313 [==============================] - 1s 2ms/step - loss: 0.5952 - acc: 0.8691


In [39]:
results

{('ResNet20v1', 'targeted-adv_ff'): {'acc_0_on_Original': 0.8452000021934509,
  'loss_0_on_Original': 0.8024741411209106,
  'acc_0_on_adv_cwl2_untargeted_ResNet56v1_1': 0.2492000013589859,
  'acc_0_on_adv_pgd_targeted_ll_ResNet20v1_1': 0.32170000672340393,
  'acc_0_on_adv_pgd_targeted_ll_ResNet56v2_2': 0.2833999991416931,
  'acc_0_on_adv_pgd_targeted_ll_ResNet20v2_2': 0.6628999710083008,
  'acc_0_on_adv_pgd_untargeted_ResNet20v2_3': 0.796500027179718,
  'acc_0_on_adv_pgd_targeted_ll_ResNet20v1_0': 0.4025999903678894,
  'acc_0_on_adv_pgd_untargeted_ResNet56v2_1': 0.8295999765396118,
  'acc_0_on_adv_pgd_targeted_ll_ResNet56v2_1': 0.7512000203132629,
  'acc_0_on_adv_pgd_targeted_ll_ResNet20v1_6': 0.46140000224113464,
  'acc_0_on_adv_pgd_untargeted_ResNet20v1_5': 0.8374999761581421,
  'acc_0_on_adv_pgd_untargeted_ResNet56v2_3': 0.15530000627040863,
  'acc_0_on_adv_pgd_untargeted_ResNet20v2_2': 0.7870000004768372,
  'acc_0_on_adv_pgd_targeted_ll_ResNet56v1_3': 0.16949999332427979,
  'acc_0_

In [117]:
results
for (structure, attack_type), eval_dict in results.items():
    accs = [eval_dict.get(f'acc_{i}', np.nan) for i in range(len(eval_dict))]
    accs.sort(reverse = True)
    acc_av = np.nanmean(accs)
    losses = [eval_dict.get(f'loss_{i}', np.nan) for i in range(len(eval_dict))]
    losses.sort()
    losses_av = np.nanmean(losses)
    row = [f'{structure}']+[f'{attack_type}'] + [acc_av] + [losses_av]
    rows.append(row)


/scratch/9181562.1.l40s/ipykernel_2063032/4189311576.py:5: RuntimeWarning: Mean of empty slice
  acc_av = np.nanmean(accs)
/scratch/9181562.1.l40s/ipykernel_2063032/4189311576.py:8: RuntimeWarning: Mean of empty slice
  losses_av = np.nanmean(losses)


NameError: name 'rows' is not defined

{'acc_0_on_Original': 0.7099999785423279,
 'loss_0_on_Original': 1.3412822484970093,
 'acc_0_on_adv_cwl2_untargeted_ResNet56v2_1': 0.10100000351667404,
 'acc_0_on_adv_pgd_targeted_ll_ResNet20v1_1': 0.12290000170469284,
 'acc_0_on_adv_pgd_targeted_ll_ResNet56v2_2': 0.12700000405311584,
 'acc_0_on_adv_pgd_targeted_ll_ResNet20v2_2': 0.1688999980688095,
 'acc_0_on_adv_pgd_untargeted_ResNet20v2_3': 0.258899986743927,
 'acc_0_on_adv_pgd_targeted_ll_ResNet20v1_0': 0.21459999680519104,
 'acc_0_on_adv_pgd_untargeted_ResNet56v2_1': 0.22269999980926514,
 'acc_0_on_adv_pgd_targeted_ll_ResNet56v2_1': 0.131400004029274,
 'acc_0_on_adv_pgd_targeted_ll_ResNet20v1_6': 0.09629999846220016,
 'acc_0_on_adv_pgd_untargeted_ResNet20v1_5': 0.2395000010728836,
 'acc_0_on_adv_pgd_untargeted_ResNet56v2_3': 0.1534000039100647,
 'acc_0_on_adv_pgd_untargeted_ResNet20v2_2': 0.2535000145435333,
 'acc_0_on_adv_pgd_targeted_ll_ResNet56v1_3': 0.0737999975681305,
 'acc_0_on_adv_pgd_targeted_ll_ResNet56v2_3': 0.0917000025

In [125]:

accs = [eval_dict.get(f'acc_{i}', np.nan) for i in range(len(eval_dict))]
accs = np.array([v for k, v in eval_dict.items() if k.startswith('acc_')])

In [127]:
accs.shape

(115,)

In [124]:
loaded_models.keys()

dict_keys(['ResNet20v1_ff_cw-adv_0.keras', 'ResNet20v1_ff_cw-adv_1.keras', 'ResNet20v1_ff_cw-adv_2.keras', 'ResNet20v1_ff_cw-adv_3.keras', 'ResNet20v1_ff_targeted-adv_0.keras', 'ResNet20v1_ff_targeted-adv_1.keras', 'ResNet20v1_ff_targeted-adv_2.keras', 'ResNet20v1_ff_targeted-adv_3.keras', 'ResNet20v1_ff_untargeted-adv_0.keras', 'ResNet20v1_ff_untargeted-adv_1.keras', 'ResNet20v1_ff_untargeted-adv_2.keras', 'ResNet20v1_ff_untargeted-adv_3.keras', 'ResNet20v2_ff_cw-adv_0.keras', 'ResNet20v2_ff_cw-adv_1.keras', 'ResNet20v2_ff_cw-adv_2.keras', 'ResNet20v2_ff_cw-adv_3.keras', 'ResNet20v2_ff_targeted-adv_0.keras', 'ResNet20v2_ff_targeted-adv_1.keras', 'ResNet20v2_ff_targeted-adv_2.keras', 'ResNet20v2_ff_targeted-adv_3.keras', 'ResNet20v2_ff_untargeted-adv_0.keras', 'ResNet20v2_ff_untargeted-adv_1.keras', 'ResNet20v2_ff_untargeted-adv_2.keras', 'ResNet20v2_ff_untargeted-adv_3.keras', 'ResNet56v1_ff_cw-adv_0.keras', 'ResNet56v1_ff_cw-adv_1.keras', 'ResNet56v1_ff_cw-adv_2.keras', 'ResNet56v1_f